In [13]:
import os

In [2]:
from torchtext.vocab import GloVe
import torch
import torchtext
from torchtext import datasets
from torchtext import data
torch.cuda.current_device()
import torch
import torchtext
from os.path import isdir
isdir("../acd")
import sys
sys.path.insert(0,"../acd")
from  dsets.sst.model import LSTMSentiment
from dsets.sst.dset import get_model, get_sst
from os import listdir
import pickle

In [3]:
torch.cuda.set_device(0)

In [4]:
def makedirs(name):
    """helper function for python 2 and 3 to call os.makedirs()
       avoiding an error if the directory to be created already exists"""

    import os, errno

    try:
        os.makedirs(name)
    except OSError as ex:
        if ex.errno == errno.EEXIST and os.path.isdir(name):
            # ignore existing directory
            pass
        else:
            # a different error happened
            raise


In [5]:
my_model = get_model("../acd/dsets/sst/sst.model")

loading ../acd/dsets/sst/sst.model


/home/lauri/.conda/envs/gpu_usage/lib/python3.6/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'model.LSTMSentiment' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/lauri/.conda/envs/gpu_usage/lib/python3.6/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


loaded onto gpu...


/home/lauri/.conda/envs/gpu_usage/lib/python3.6/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.LSTM' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/lauri/.conda/envs/gpu_usage/lib/python3.6/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [10]:
inputs = data.Field(lower=True)
answers = data.Field(sequential=False, unk_token=None)


In [11]:
train, dev, test = datasets.SST.splits(inputs, answers, fine_grained=False, train_subtrees=True,
                                       filter_pred=lambda ex: ex.label != 'neutral')

inputs.build_vocab(train, dev, test)

In [14]:
if os.path.isfile(os.path.join(os.getcwd(), '.vector_cache/input_vectors.pt')):
    inputs.vocab.vectors = torch.load(os.path.join(os.getcwd(), '.vector_cache/input_vectors.pt'))
else:
    inputs.vocab.load_vectors("glove.6B.300d")
    makedirs(os.path.dirname(os.path.join(os.getcwd(), '.vector_cache/input_vectors.pt')))
    torch.save(inputs.vocab.vectors, os.path.join(os.getcwd(), '.vector_cache/input_vectors.pt'))
answers.build_vocab(train)

train_iter, dev_iter, test_iter = data.BucketIterator.splits(
    (train, dev, test), batch_size=50, device=torch.device(0))

In [16]:
my_model.eval()

LSTMSentiment(
  (embed): Embedding(18844, 300)
  (lstm): LSTM(300, 128)
  (hidden_to_label): Linear(in_features=128, out_features=2, bias=True)
)

In [17]:
dev_iter.init_epoch()

In [39]:
torch.device(0)

device(type='cuda', index=0)

In [36]:
n_dev_correct, dev_loss = 0, 0
for dev_batch_idx, dev_batch in enumerate(dev_iter):
    answer = my_model(dev_batch)
    n_dev_correct += (
        torch.max(answer, 1)[1].view(dev_batch.label.size()).data == dev_batch.label.data).sum()
    #dev_loss = criterion(answer, dev_batch.label)
dev_acc = 100. * n_dev_correct / len(dev)